# Import Dataset

In [0]:
import pandas as pd
import numpy as np
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> l
Packages:
  [ ] abc................. Australian Broadcasting Commission 2006
  [ ] alpino.............. Alpino Dutch Treebank
  [ ] averaged_perceptron_tagger Averaged Perceptron Tagger
  [ ] averaged_perceptron_tagger_ru Averaged Perceptron Tagger (Russian)
  [ ] basque_grammars..... Grammars for Basque
  [ ] biocreative_ppi..... BioCreAtIvE (Critical Assessment of Information
                           Extraction Systems in Biology)
  [ ] bllip_wsj_no_aux.... BLLIP Parser: WSJ Model
  [ ] book_grammars....... Grammars from NLTK Book
  [ ] brown............... Brown Corpus
  [ ] brown_tei........... Brown Corpus (TEI XML Version)
  [ ] cess_cat............ CESS-CAT Treebank
  [

KeyboardInterrupt: ignored

In [0]:
data = pd.read_excel('/content/drive/Shared drives/OPIM 5671 B12 Team one/Dataset/Combined_News_DJIA.xls')

In [0]:
data.shape

(1989, 27)

In [0]:
data.head(277)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,Top11,Top12,Top13,Top14,Top15,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"""Georgia 'downs two Russian warplanes' as cou...",'BREAKING: Musharraf to be impeached.','Russia Today: Columns of troops roll into So...,'Russian tanks are moving towards the capital...,"""Afghan children raped with 'impunity,' U.N. ...",'150 Russian tanks have entered South Ossetia...,"""Breaking: Georgia invades South Ossetia, Rus...","""The 'enemy combatent' trials are nothing but...",'Georgian troops retreat from S. Osettain cap...,'Did the U.S. Prep Georgia for War with Russia?','Rice Gives Green Light for Israel to Attack ...,'Announcing:Class Action Lawsuit on Behalf of...,"""So---Russia and Georgia are at war and the N...","""China tells Bush to stay out of other countr...",'Did World War III start today?','Georgia Invades South Ossetia - if Russia ge...,'Al-Qaeda Faces Islamist Backlash',"'Condoleezza Rice: ""The US would not act to p...",'This is a busy day: The European Union has ...,"""Georgia will withdraw 1,000 soldiers from Ir...",'Why the Pentagon Thinks Attacking Iran is a ...,'Caucasus in crisis: Georgia invades South Os...,'Indian shoe manufactory - And again in a se...,'Visitors Suffering from Mental Illnesses Ban...,"""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,'Why wont America and Nato help us? If they w...,'Bush puts foot down on Georgian conflict',"""Jewish Georgian minister: Thanks to Israeli ...",'Georgian army flees in disarray as Russians ...,"""Olympic opening ceremony fireworks 'faked'""",'What were the Mossad with fraudulent New Zea...,'Russia angered by Israeli military sale to G...,'An American citizen living in S.Ossetia blam...,'Welcome To World War IV! Now In High Definit...,"""Georgia's move, a mistake of monumental prop...",'Russia presses deeper into Georgia; U.S. say...,'Abhinav Bindra wins first ever Individual Ol...,' U.S. ship heads for Arctic to define territ...,'Drivers in a Jerusalem taxi station threaten...,'The French Team is Stunned by Phelps and the...,'Israel and the US behind the Georgian aggres...,"'""Do not believe TV, neither Russian nor Geor...",'Riots are still going on in Montreal (Canada...,'China to overtake US as largest manufacturer','War in South Ossetia [PICS]','Israeli Physicians Group Condemns State Tort...,' Russia has just beaten the United States ov...,'Perhaps *the* question about the Georgia - R...,'Russia is so much better at war',"""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,'Remember that adorable 9-year-old who sang a...,"""Russia 'ends Georgia operation'""","'""If we had no sexual harassment we would hav...","""Al-Qa'eda is losing support in Iraq because ...",'Ceasefire in Georgia: Putin Outmaneuvers the...,'Why Microsoft and Intel tried to kill the XO...,'Stratfor: The Russo-Georgian War and the Bal...,"""I'm Trying to Get a Sense of This Whole Geor...","""The US military was surprised by the timing ...",'U.S. Beats War Drum as Iran Dumps the Dollar',"'Gorbachev: ""Georgian military attacked the S...",'CNN use footage of Tskhinvali ruins to cover...,'Beginning a war as the Olympics were opening...,'55 pyramids as large as the Luxor stacked in...,'The 11 Top Party Cities in the World','U.S. troops still in Georgia (did you know t...,'Why Russias response to Georgia was right',"'Gorbachev accuses U.S. of making a ""serious ...","'Russia, Georgia, and NATO: Cold War Two'",'Remember that adorable 62-year-old who led y...,'War in Georgia: The Israeli connection','All signs point to the US encouraging Georgi...,'Christopher King argues that the US and NATO...,'America: The New Mexico?',"""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,' U.S. refuses Israel weapons to attack Iran:...,"""When the president ordered to attack Tskhinv...",' Israel clears troops who killed Reuters cam...,'Britain\'s policy of being tough on drugs is...,'Body of

In [0]:
data['text'] = data['Top1'] +  data['Top2'] + data['Top3'] + data['Top4'] + data['Top5'] + data['Top6'] + data['Top7'] + data['Top8'] + data['Top9'] + data['Top10'] + data['Top11'] + data['Top12'] + data['Top13'] + data['Top14'] + data['Top15'] + data['Top16'] + data['Top17'] + data['Top18'] + data['Top19'] + data['Top20'] + data['Top21'] + data['Top22'] + data['Top23'] + data['Top24'] + data['Top25']

In [0]:
data['text']

0        "Georgia 'downs two Russian warplanes' as cou...
1        'Why wont America and Nato help us? If they w...
2        'Remember that adorable 9-year-old who sang a...
3        ' U.S. refuses Israel weapons to attack Iran:...
4        'All the experts admit that we should legalis...
                              ...                        
1984    Barclays and RBS shares suspended from trading...
1985    2,500 Scientists To Australia: If You Want To ...
1986    Explosion At Airport In IstanbulYemeni former ...
1987    Jamaica proposes marijuana dispensers for tour...
1988    A 117-year-old woman in Mexico City finally re...
Name: text, Length: 1989, dtype: object

# Text Preprocessing

In [0]:
list_rows = []

In [0]:
for index, row in data.iterrows():
    # split into words by white space
    words = row['text'].split()
    
    # remove punctuation from each word
    import string
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in words]
    
    # convert to lower case
    stripped = [word.lower() for word in stripped]
    
    # Filter out stop words
    from nltk.corpus import stopwords
    stopwords = set(stopwords.words('english'))
    removed_stop_words = [w for w in stripped if not w in stopwords]
    
    # Stemming of words 单词不太好看？
    from nltk.stem.porter import PorterStemmer
    porter = PorterStemmer()
    stemmed = [porter.stem(word) for word in removed_stop_words]
    list_rows.append(stemmed)

LookupError: ignored

In [0]:
data['clean'] = list_rows
data

ValueError: ignored

# Sentiment Analysis

In [0]:
from textblob import TextBlob

pol = lambda x: TextBlob(x).sentiment.polarity
sub = lambda x: TextBlob(x).sentiment.subjectivity

data['polarity'] = data['clean'].apply(pol)
data['subjectivity'] = data['clean'].apply(sub)
data

In [0]:
# Visualize polarity and subjectivity
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [10, 8]

for index, date in enumerate(data.index):
    x = data['Date']
    y_polarity = data['polarity']
    y_subjectivity = data['subjectivity']
    plt.plot(x, y_polarity, color='blue')
    plt.plot(x, y_subjectivity, color='red')
    
plt.title('Sentiment Analysis', fontsize=20)
plt.xlabel('Date', fontsize=15)
plt.ylabel('Ratio', fontsize=15)

plt.show()

# Topic Modeling - Latent Dirichlet Allocation (LDA)

In [0]:
# Create a function to pull out nouns from a string of text
from nltk import word_tokenize, pos_tag

# Let's create a function to pull out nouns from a string of text
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

In [0]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns_adj = data['text'].apply(nouns_adj)
data_nouns_adj

In [0]:
# Import the necessary modules for LDA with gensim
# Terminal / Anaconda Navigator: conda install -c conda-forge gensim
from gensim import matutils, models
import scipy.sparse

In [0]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
                  'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said','new',
                  'say','says','said','first','second','will','year','years','now','one','may','just']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
cvna = CountVectorizer(stop_words=stop_words, max_df=.8) #n-gram?
data_cvna = cvna.fit_transform(data_nouns_adj)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names())
data_dtmna.index = data_nouns_adj.index
data_dtmna

# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [0]:
# Try 20 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=20, id2word=id2wordna, passes=200)
ldana.print_topics()

# Visualizing the models with pyLDAvis

In [0]:
from sklearn.decomposition import LatentDirichletAllocation
lda_tf = LatentDirichletAllocation(n_components=20,random_state=0)
lda_tf.fit(data_cvna)

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_tf, data_cvna, cvna)

# TF-IDF Vectorizing & Singular Value Decomposition(SVD)

## Use cleaned text¶

In [0]:
# https://www.kaggle.com/ndrewgele/omg-nlp-with-the-djia-and-reddit
train_word = []
for row in range(0,len(train.index)):
    train_word.append(' '.join(str(x) for x in train.loc[row,'clean']))

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(use_idf = True) #Do we need ngram_range=(1,3)? 
X = vectorizer.fit_transform(train_word)

X[0]
X.shape

In [0]:
from sklearn.decomposition import TruncatedSVD

lsa = TruncatedSVD(n_components=1611, n_iter=100)
lsa.fit(X)

In [0]:
terms = vectorizer.get_feature_names()
for i, comp in enumerate(lsa.components_):
    termsInComp = zip(terms, comp)
    sortedTerms = sorted(termsInComp, key=lambda x: x[1], reverse=True)[:10]
    print("Concept %d:" %i)
    for term in sortedTerms:
        print(term[0])
    print(" ")

# Partitioning the data

In [0]:
train = data[data['Date'] < '2015-01-01']
test = data[data['Date'] > '2014-12-31']